## Importing Libraries:

In [ ]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn.model_selection import cross_validate
from sklearn import tree
from sklearn import svm
from sklearn import linear_model
import csv
from sklearn import metrics
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
import gzip
import zipfile

## Creating Data Processing Function

In [ ]:
def loadData(df,scaler=None):
    data = pd.DataFrame(index=range(len(df)))
    
    data = df.get(['X','Y']) #2
    
    #data = data.join(pd.get_dummies(df['DayOfWeek'])) #7
    DayOfWeeks = df.DayOfWeek.unique()
    DayOfWeekMap = {}
    i = 0
    for s in DayOfWeeks:
        DayOfWeekMap[s] = i
        i += 1
    data = data.join(df['DayOfWeek'].map(DayOfWeekMap))
    
    #data = data.join(pd.get_dummies(df['PdDistrict'])) #10
    PdDistricts = df.PdDistrict.unique()
    PdDistrictMap = {}
    i = 0
    for s in PdDistricts:
        PdDistrictMap[s] = i
        i += 1
    data = data.join(df['PdDistrict'].map(PdDistrictMap))
        
    date_time = pd.to_datetime(df.Dates)
    year = date_time.dt.year
    data['Year'] = year
    month = date_time.dt.month
    data['Month'] = month
    day = date_time.dt.day
    data['Day'] = day
    hour = date_time.dt.hour
    data['hour'] = hour
    minute = date_time.dt.minute
    time = hour*60+minute
    data['Time'] = time
    #data = data.join(pd.get_dummies(year)) #13
    #data = data.join(pd.get_dummies(month)) #12
    
    data['StreetCorner'] = df['Address'].str.contains('/').map(int)
    data['Block'] = df['Address'].str.contains('Block').map(int)
    
    X = data.values
    Y = None
    if 'Category' in df.columns:
        Y = df.Category.values
    
    #scaler = preprocessing.StandardScaler().fit(data.values)
    #X = scaler.transform(data.values)
    return X,Y,scaler


## Creating Function for Gaussian Naive-Bayes

In [5]:
def NB(X,Y):
    clf = GaussianNB()
    #clf = MultinomialNB()
    #clf = BernoulliNB()
    scoreSum = 0
    kfold = cross_validation.KFold(len(X),n_folds=3)
    for train, test in kfold:
        #clf.set_params(max_features=max_features[0])
        clf.fit(X[train],Y[train])
        Yhat = clf.predict_proba(X[test])
        score = metrics.log_loss(Y[test],Yhat)
        print(score)
        scoreSum += score
    print(scoreSum/4.0)
    return clf

def NBpredict(X,Y,Xhat):
    clf = GaussianNB()
    clf.fit(X,Y)
    Yhat = clf.predict_proba(Xhat)
    return Yhat,clf

In [ ]:
train = pd.read_csv("./train.csv")
X,Y,scaler = loadData(train)

test = pd.read_csv("./test.csv")
Xhat,_,__ = loadData(test,scaler)
Yhat,clf = NBpredict(X,Y,Xhat)

submission = pd.DataFrame(Yhat,columns=clf.classes_)
submission['Id'] = test.Id.tolist()
submission.to_csv(gzip.open('GNB.csv.gz','wt'),index=False)

## Result: 
Logloss Score: 3.888

In [ ]:
def MNB(X,Y):
    clf = GaussianNB()
    #clf = MultinomialNB()
    #clf = BernoulliNB()
    scoreSum = 0
    kfold = cross_validation.KFold(len(X),n_folds=3)
    for train, test in kfold:
        #clf.set_params(max_features=max_features[0])
        clf.fit(X[train],Y[train])
        Yhat = clf.predict_proba(X[test])
        score = metrics.log_loss(Y[test],Yhat)
        print(score)
        scoreSum += score
    print(scoreSum/4.0)
    return clf

def MNBpredict(X,Y,Xhat):
    clf = MultinomialNB()
    clf.fit(X,Y)
    Yhat = clf.predict_proba(Xhat)
    return Yhat,clf